In [ ]:
import os
import sys
os.environ["THEANO_FLAGS"] = "device=gpu"
sys.path.append(os.getcwd()+"/..")
from tasks import check

In [ ]:
from model import model53, model54

In [ ]:
all_layers = model54(nb_filters=512, w=32, h=32, c=3)

In [ ]:
import theano.tensor as T
import theano
from lasagne import layers

X_scale1 = T.tensor4()
X_scale2 = T.tensor4()
X_scale3 = T.tensor4()
X_scale4 = T.tensor4()

X_scale1_out = T.tensor4()
X_scale2_out = T.tensor4()
X_scale3_out = T.tensor4()
X_scale4_out = T.tensor4()


input_tensors = [X_scale4]
output_tensors = [X_scale1]

input_layers = ["input"]
output_layers = ["output"]

inputs = {all_layers[l]: t for t, l in zip(input_tensors, input_layers)}
outputs = map(lambda name:all_layers[name], output_layers)

output_pred_tensors = layers.get_output(outputs, inputs)
lambdas = [1]
loss = 0
for outp, outp_pred, lambda_ in zip(output_tensors, output_pred_tensors, lambdas):
    loss += ((outp_pred - outp) ** 2).mean() * lambda_

In [ ]:
from lasagne import updates
import numpy as np
params = layers.get_all_params(all_layers.values(), trainable=True)
print("Number of params : {}".format(layers.count_params(all_layers.values(), trainable=True))) 
lr = theano.shared(np.array(0.1, dtype=theano.config.floatX))
params_updates = updates.adam(loss, params, learning_rate=lr)

In [ ]:
from lasagnekit.misc.draw_net import draw_to_file
from IPython.display import SVG
draw_to_file(layers.get_all_layers(all_layers['output']), 'out.svg')
SVG('out.svg')

In [ ]:
train_fn = theano.function(input_tensors + output_tensors, loss, updates=params_updates, on_unused_input='warn')

In [ ]:
predict_fn = theano.function(input_tensors, output_pred_tensors, on_unused_input='warn')

In [ ]:
from data import load_data
from lasagnekit.datasets.rescaled import Rescaled
from lasagnekit.easy import iterate_minibatches
from lasagnekit.datasets.imagecollection import ImageCollection
from lasagnekit.datasets.transformed import Transformed
w, h = 32, 32
scale = {1: 32, 2: 16, 3: 8, 4: 4}
batch_size = 128

c = 3
folder = "{}/icons".format(os.getenv("DATA_PATH"))
data_scale1 = ImageCollection(size=(w, h), nb=1535, mode='all', folder=folder)

def preprocess(x, s=32):
    x = x.reshape((x.shape[0], s, s, c))
    x = x.transpose((0, 3, 1, 2))
    return x
data_scale1.load()

In [ ]:
print(all_layers['output'].output_shape)

In [ ]:
lr.set_value(0.01)
loss_list = []
for i in range(10000):
        
    X_scale1 = preprocess(data_scale1.X, s=scale[1])

    total_loss = 0.
    for mini_batch in iterate_minibatches(len(data_scale1.X), batch_size):
        loss_val = train_fn(X_scale1[mini_batch], X_scale1[mini_batch])
        total_loss += loss_val
    print(total_loss)
    loss_list.append(total_loss)
plt.plot(loss_list)

In [ ]:
out1, = predict_fn(X_scale1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color
fig = plt.figure(figsize=(10, 10))
img = dispims_color(out1[0:100].transpose((0, 2, 3, 1)),border=1)
plt.imshow(img, interpolation='none')

In [ ]:
ss = np.random.uniform(size=(100, c, 32, 32))
ss = ss.astype(np.float32)
for i in range(100):
    ss, = predict_fn(ss)
    
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color
fig = plt.figure(figsize=(10, 10))
img = dispims_color(ss[0:100].transpose((0, 2, 3, 1)),border=1)
plt.axis('off')
plt.imshow(img, interpolation='none')